This notebook's purpose is to solve the "Klotski puzzle" using a script.

Several different-sized block pieces are placed inside a box, which is in 4×5 size. Among the blocks, there is a special one (2x2 red square) which must be moved to the exit (located at the center of the bottom edge of the box). 

The player is not allowed to remove blocks, and may only slide blocks horizontally and vertically. Common goals are to solve the puzzle with a minimum number of moves or in a minimum amount of time.

The initial state of the the box and pieces is represented below:

<img src="klotski-img.png">

In [ ]:
import random
import copy
import numpy as np

In [ ]:
WIDTH = 4
HEIGHT = 5
can_move_down = ["0", "^"]
can_move_left = ["0", ">"]
can_move_right = ["0", "<"]
can_move_up = ["0", "v"]
can_move_down_if_right_empty = [">", "b"]
can_move_down_if_left_empty = ["<", "d"]
can_move_up_if_right_empty = [">", "p"]
can_move_up_if_left_empty = ["<", "q"]
can_move_left_if_top_empty = ["^", "b"]
can_move_left_if_bottom_empty = ["v", "p"]
can_move_right_if_top_empty = ["^", "d"]
can_move_right_if_bottom_empty = ["v", "q"]

We store the description of the inital state in a Numpy (prints better than vanilla 2-dimensional Python arrays) array. 

In [ ]:
START = np.array([
    ["v", "p", "q", "v"],
    ["^", "b", "d", "^"],
    ["v", ">", "<", "v"],
    ["^", "0", "0", "^"],
    ["0", ".", ".", "0"]])

We chose characters so as to represent the different pieces. One has to imagine the lines that connect characters together!

<img src="klotski_characters.png">

In [ ]:
counter = 0
steps =[]
current_deck = START
# Shallow copy to actually store current deck state, and avoid having it change alongside variable
steps.append(copy.copy(current_deck))
empty_cell_data = {}

We start by defining the functions we'll be using during each iteration:

In [ ]:
def initialize_cell_data(cell_data):
    cell_data = {
    'row': "?",
    'col': "?",
    'bottom': ["?",{"can_move": False}],
    'top': ["?",{"can_move": False}],
    'left': ["?",{"can_move": False}],
    'right': ["?",{"can_move": False}],
    'has_movable_neighbour': False
    }
    return cell_data

In [ ]:
def get_rand_empty_cell(deck):
    #find the two empty cells
    empty_cells = []
    for i in range(0, HEIGHT):
        for j in range(0, WIDTH):
            if(deck[i, j]) == ".":
                empty_cells.append([i, j])
    
    #pick one
    empty_cell_coords = empty_cells[random.randint(0, 1)]
    return empty_cell_coords

In [ ]:
def enrich_cell_data(deck, cell_data):
    # describe content of nearby cells
    if cell_data["row"] == 0:
        cell_data["top"][0] = "out of deck"
    else:
        cell_data["top"][0] = deck[cell_data["row"]-1, cell_data["col"]]

    if cell_data["row"] == HEIGHT-1:
        cell_data["bottom"][0] = "out of deck"
    else:
        
        cell_data["bottom"][0] = deck[cell_data["row"]+1, cell_data["col"]]

    if cell_data["col"] == 0:
        cell_data["left"][0] = "out of deck"
    else:
        cell_data["left"][0] = deck[cell_data["row"], cell_data["col"]-1]

    if cell_data["col"] == WIDTH-1:
        cell_data["right"][0] = "out of deck"
    else:
        cell_data["right"][0] = deck[cell_data["row"], cell_data["col"]+1]
    
    # check if nearby cells contain pieces that can be moves into the empty cell
    if cell_data["top"][0] in can_move_down:
        cell_data["top"][1]["can_move"] = True
    elif cell_data["top"][0] in can_move_down_if_left_empty and cell_data["left"][0] == ".":
        cell_data["top"][1]["can_move"] = True
    elif cell_data["top"][0] in can_move_down_if_right_empty and cell_data["right"][0] == ".":
        cell_data["top"][1]["can_move"] = True
    
    if cell_data["bottom"][0] in can_move_up:
            cell_data["bottom"][1]["can_move"] = True
    elif cell_data["bottom"][0] in can_move_up_if_left_empty and cell_data["left"][0] == ".":
            cell_data["bottom"][1]["can_move"] = True
    elif cell_data["bottom"][0] in can_move_down_if_right_empty and cell_data["right"][0] == ".":
            cell_data["bottom"][1]["can_move"] = True
    
    if cell_data["left"][0] in can_move_right:
            cell_data["left"][1]["can_move"] = True
    elif cell_data["left"][0] in can_move_right_if_top_empty and cell_data["top"][0] == ".":
            cell_data["left"][1]["can_move"] = True
    elif cell_data["left"][0] in can_move_right_if_bottom_empty and cell_data["bottom"][0] == ".":
            cell_data["left"][1]["can_move"] = True

    if cell_data["right"][0] in can_move_left:
            cell_data["right"][1]["can_move"] = True
    elif cell_data["right"][0] in can_move_left_if_top_empty and cell_data["top"][0] == ".":
            cell_data["right"][1]["can_move"] = True
    elif cell_data["right"][0] in can_move_left_if_bottom_empty and cell_data["bottom"][0] == ".":
            cell_data["right"][1]["can_move"] = True
    
    cell_data["has_movable_neighbour"] = cell_data["top"][1]["can_move"] or cell_data["right"][1]["can_move"] or cell_data["bottom"][1]["can_move"] or cell_data["left"][1]["can_move"]
    
    return cell_data

In [ ]:
def movable_neighbour_chosen(deck, cell_data):
    # check nearby cells which content can be moved
    movable_neighbours = []
    for key in ["top", "bottom", "left", "right"]:
        if cell_data[key][1]["can_move"]:
            movable_neighbours.append(key)

    # pick one
    return random.choice(movable_neighbours)

In [ ]:
def move(deck, cell_data, neighbour_to_be_moved):
    # we start by checking position of piece to move relative to empty cell selected
    if neighbour_to_be_moved == "top":
        # if piece to move is a 1x1 square        
        if cell_data["top"][0] == "0":
            deck[cell_data["row"] - 1, cell_data["col"]] = "."
            deck[cell_data["row"], cell_data["col"]] = "0"
        # if piece to move is a vertical rectangle
        elif cell_data["top"][0] == "^":
            deck[cell_data["row"] - 2, cell_data["col"]] = "."
            deck[cell_data["row"] - 1, cell_data["col"]] = "v"
            deck[cell_data["row"], cell_data["col"]] = "^"
        # if piece to move is a horizontal rectangle
        elif cell_data["top"][0] == ">":
            deck[cell_data["row"] - 1, cell_data["col"]] = "."
            deck[cell_data["row"] - 1, cell_data["col"] + 1] = "."
            deck[cell_data["row"], cell_data["col"]] = ">"
            deck[cell_data["row"], cell_data["col"] + 1] = "<"
        elif cell_data["top"][0] == "<":
            deck[cell_data["row"] - 1, cell_data["col"] - 1] = "."
            deck[cell_data["row"] - 1, cell_data["col"]] = "."
            deck[cell_data["row"], cell_data["col"] - 1] = ">"
            deck[cell_data["row"], cell_data["col"]] = "<"
        # if piece to move is a 2x2 square
        elif cell_data["top"][0] == "b":
            deck[cell_data["row"] - 2, cell_data["col"]] = "."
            deck[cell_data["row"] - 2, cell_data["col"] + 1] = "."
            deck[cell_data["row"] - 1, cell_data["col"]] = "p"
            deck[cell_data["row"] - 1, cell_data["col"] + 1] = "q"
            deck[cell_data["row"] , cell_data["col"]] = "b"
            deck[cell_data["row"], cell_data["col"] + 1] = "d"
        elif cell_data["top"][0] == "d":
            deck[cell_data["row"] - 2, cell_data["col"] - 1] = "."
            deck[cell_data["row"] - 2, cell_data["col"]] = "."
            deck[cell_data["row"] - 1, cell_data["col"] - 1] = "p"
            deck[cell_data["row"] - 1, cell_data["col"]] = "q"
            deck[cell_data["row"] , cell_data["col"] - 1] = "b"
            deck[cell_data["row"], cell_data["col"]] = "d"

    elif neighbour_to_be_moved == "left":
        # if piece to move is a 1x1 square
        if cell_data["left"][0] == "0":
            deck[cell_data["row"], cell_data["col"] - 1] = "."
            deck[cell_data["row"], cell_data["col"]] = "0"
        # if piece to move is a horizontal rectangle
        elif cell_data["left"][0] == "<":
            deck[cell_data["row"], cell_data["col"] - 2] = "."
            deck[cell_data["row"], cell_data["col"] - 1] = ">"
            deck[cell_data["row"], cell_data["col"]] = "<"
        # if piece to move is a vertical rectangle
        elif cell_data["left"][0] == "^":
            deck[cell_data["row"] - 1, cell_data["col"]] = "v"
            deck[cell_data["row"], cell_data["col"]] = "^"
            deck[cell_data["row"] - 1, cell_data["col"] - 1] = "."
            deck[cell_data["row"], cell_data["col"] - 1] = "."
        elif cell_data["left"][0] == "v":
            deck[cell_data["row"], cell_data["col"]] = "v"
            deck[cell_data["row"] + 1, cell_data["col"]] = "^"
            deck[cell_data["row"], cell_data["col"] - 1] = "."
            deck[cell_data["row"] + 1, cell_data["col"] - 1] = "."
        # if piece to move is a 2x2 square
        elif cell_data["left"][0] == "d":
            deck[cell_data["row"] - 1, cell_data["col"] - 2] = "."
            deck[cell_data["row"], cell_data["col"] - 2] = "."
            deck[cell_data["row"] - 1, cell_data["col"] - 1] = "p"
            deck[cell_data["row"], cell_data["col"] - 1] = "b"
            deck[cell_data["row"] - 1, cell_data["col"]] = "q"
            deck[cell_data["row"], cell_data["col"]] = "d"
        elif cell_data["left"][0] == "q":
            deck[cell_data["row"], cell_data["col"] - 2] = "."
            deck[cell_data["row"] + 1, cell_data["col"] - 2] = "."
            deck[cell_data["row"], cell_data["col"] - 1] = "p"
            deck[cell_data["row"] + 1, cell_data["col"] - 1] = "b"
            deck[cell_data["row"], cell_data["col"]] = "q"
            deck[cell_data["row"] + 1, cell_data["col"]] = "d"

    elif neighbour_to_be_moved == "bottom":
        # if piece to move is a 1x1 square
        if cell_data["bottom"][0] == "0":
            deck[cell_data["row"] + 1, cell_data["col"]] = "."
            deck[cell_data["row"], cell_data["col"]] = "0"
        # if piece to move is a vertical rectangle
        elif cell_data["bottom"][0] == "v":
            deck[cell_data["row"], cell_data["col"]] = "v"
            deck[cell_data["row"] + 1, cell_data["col"]] = "^"
            deck[cell_data["row"] + 2, cell_data["col"]] = "."
        # if piece to move is a horizontal rectangle
        elif cell_data["bottom"][0] == ">":
            deck[cell_data["row"], cell_data["col"]] = ">"
            deck[cell_data["row"], cell_data["col"] + 1] = "<"
            deck[cell_data["row"] + 1, cell_data["col"]] = "."
            deck[cell_data["row"] + 1, cell_data["col"] + 1] = "."
        elif cell_data["bottom"][0] == "<":
            deck[cell_data["row"], cell_data["col"] - 1] = ">"
            deck[cell_data["row"], cell_data["col"]] = "<"
            deck[cell_data["row"] + 1, cell_data["col"] - 1] = "."
            deck[cell_data["row"] + 1, cell_data["col"]] = "."
        # if piece to move is a 2x2 square
        elif cell_data["bottom"][0] == "p":
            deck[cell_data["row"], cell_data["col"]] = "p"
            deck[cell_data["row"], cell_data["col"] + 1] = "q"
            deck[cell_data["row"] + 1, cell_data["col"]] = "b"
            deck[cell_data["row"] + 1, cell_data["col"] + 1] = "d"
            deck[cell_data["row"] + 2, cell_data["col"]] = "."
            deck[cell_data["row"] + 2, cell_data["col"] + 1] = "."
        elif cell_data["bottom"][0] == "q":
            deck[cell_data["row"], cell_data["col"] - 1] = "p"
            deck[cell_data["row"], cell_data["col"]] = "q"
            deck[cell_data["row"] + 1, cell_data["col"] - 1] = "b"
            deck[cell_data["row"] + 1, cell_data["col"]] = "d"
            deck[cell_data["row"] + 2, cell_data["col"] - 1] = "."
            deck[cell_data["row"] + 2, cell_data["col"]] = "."

    elif neighbour_to_be_moved == "right":
        # if piece to move is a 1x1 square        
        if cell_data["right"][0] == "0":
            deck[cell_data["row"], cell_data["col"]] = "0"
            deck[cell_data["row"], cell_data["col"] + 1] = "."
        # if piece to move is a horizontal rectangle
        elif cell_data["right"][0] == ">":
            deck[cell_data["row"], cell_data["col"]] = ">"
            deck[cell_data["row"], cell_data["col"] + 1] = "<"
            deck[cell_data["row"], cell_data["col"] + 2] = "."
        # if piece to move is a vertical rectangle
        elif cell_data["right"][0] == "^":
            deck[cell_data["row"] - 1, cell_data["col"]] = "v"
            deck[cell_data["row"], cell_data["col"]] = "^"
            deck[cell_data["row"] - 1, cell_data["col"] + 1] = "."
            deck[cell_data["row"], cell_data["col"] + 1] = "."
        elif cell_data["right"][0] == "v":
            deck[cell_data["row"], cell_data["col"]] = "v"
            deck[cell_data["row"] + 1, cell_data["col"]] = "^"
            deck[cell_data["row"], cell_data["col"] + 1] = "."
            deck[cell_data["row"] + 1, cell_data["col"] + 1] = "."
        # if piece to move is a 2x2 square
        elif cell_data["right"][0] == "b":
            deck[cell_data["row"] - 1, cell_data["col"]] = "p"
            deck[cell_data["row"], cell_data["col"]] = "b"
            deck[cell_data["row"] - 1, cell_data["col"] + 1] = "q"
            deck[cell_data["row"], cell_data["col"] + 1] = "d"
            deck[cell_data["row"] - 1, cell_data["col"] + 2] = "."
            deck[cell_data["row"], cell_data["col"] + 2] = "."
        elif cell_data["right"][0] == "p":
            deck[cell_data["row"], cell_data["col"]] = "p"
            deck[cell_data["row"] + 1, cell_data["col"]] = "b"
            deck[cell_data["row"], cell_data["col"] + 1] = "q"
            deck[cell_data["row"] + 1, cell_data["col"] + 1] = "d"
            deck[cell_data["row"], cell_data["col"] + 2] = "."
            deck[cell_data["row"] + 1, cell_data["col"] + 2] = "."

    return deck

We can now start solving:

In [ ]:
print(steps[0])
print()

# Counter to count the iterations
i = 0

# While loop inside of which piece movements are carried out
while True:
    empty_cell_data = initialize_cell_data(empty_cell_data)
    
    # If we pick an empty cell whose neighbour pieces cannot be moved into it, we need to pick the other empty cell
    while not empty_cell_data["has_movable_neighbour"]:
        empty_cell_coords = get_rand_empty_cell(current_deck)

        empty_cell_data["row"] = empty_cell_coords[0]
        empty_cell_data["col"] = empty_cell_coords[1]

        empty_cell_data = enrich_cell_data(current_deck, empty_cell_data)

    print(f"Current cell environment is: {empty_cell_data}")
    neighbour_to_be_moved = movable_neighbour_chosen(current_deck, empty_cell_data)
    
    print()
    print(f"{neighbour_to_be_moved} is the position of the element we'll move")

    current_deck = move(current_deck, empty_cell_data, neighbour_to_be_moved)

    steps.append(copy.copy(current_deck))
    
    print()
    print(f"Iteration #{i+1}: after last move, deck has become:")
    print(steps[i+1])
    i += 1
    print()
    
    # We monitor the position of the bottom-left corner of the 2x2 square, and stop looping when it has arrived at the exit
    if np.where(current_deck == "b")[0][0] == 4 and np.where(current_deck == "b")[1][0] == 1:
        print(f"Victory! We solved the Klotski puzzle after {i} iterations!!!")
        break